# RNN and Transformers

In this lab lesson we will see how and when to use Recurrent Neural Networks and how to exploits pre-trained Transformers model like Bert.

We will be trying to create a model which is able to predict if a sentence contains subjectivity or it is fully objective.

In [1]:
# Import necessary libraries

import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight 

In [2]:
# Load the data

# Use this utility function to load the data
def  get_data_split(split):
  """
  Args:
    - split: the split of the data you want to load.
  Returns:
    - X, y data, where X is the array containing the sentences and y is the labels vector.

  """
  df = pd.read_csv(f'sentences{split.capitalize()}.csv')
  return df['FRASE'].values, df['TAG_FRASE'].values


In [3]:
sentences_train, labels_train = get_data_split(split = 'train')
sentences_val, labels_val = get_data_split(split = 'val')
sentences_test, labels_test = get_data_split(split = 'test')

## RNN

Recurrent neural networks (RNN) are a class of neural networks that is powerful for
modeling sequence data such as time series or natural language.

Schematically, a RNN layer uses a `for` loop to iterate over the timesteps of a
sequence, while maintaining an internal state that encodes information about the timesteps it has seen so far.

The Keras RNN API is designed with a focus on:

- **Ease of use**: the built-in `keras.layers.RNN`, `keras.layers.LSTM`,
`keras.layers.GRU` layers enable you to quickly build recurrent models without
having to make difficult configuration choices.

- **Ease of customization**: You can also define your own RNN cell layer (the inner
part of the `for` loop) with custom behavior, and use it with the generic
`keras.layers.RNN` layer (the `for` loop itself). This allows you to quickly
prototype different research ideas in a flexible way with minimal code.

There are three built-in RNN layers in Keras:

1. `keras.layers.SimpleRNN`, a fully-connected RNN where the output from previous
timestep is to be fed to next timestep.

2. `keras.layers.GRU`
3. `keras.layers.LSTM`

### Data pre-processing

In [4]:
# Tokenization: vectorize a text corpus, by turning each text into 
# either a sequence of integers (each integer being the index of 
# a token in a dictionary) or into a vector where the coefficient 
# for each token could be binary.

from tensorflow.keras.preprocessing.text import Tokenizer

label_to_idx_dict = {'OGG' : 0, 'SOG' : 1}
label_to_idx_f = np.vectorize(label_to_idx_dict.get)

vocab_dim = 10000

def get_tokenizer(x_train):
  tokenizer = Tokenizer(num_words = vocab_dim)
  tokenizer.fit_on_texts(x_train)
  return tokenizer

tokenizer = get_tokenizer(sentences_train)

In [5]:
# pad_sequences :transforms a list (of length num_samples) 
# of sequences (lists of integers) into a 2D Numpy array of 
# shape (num_samples, num_timesteps)

from tensorflow.keras.utils import pad_sequences

maxSentenceLen = 20

generate_x = lambda x: pad_sequences(tokenizer.texts_to_sequences(x), maxlen = maxSentenceLen, padding = 'post')

x_train = generate_x(sentences_train)
x_test = generate_x(sentences_test)
x_val = generate_x(sentences_val)

y_train = label_to_idx_f(labels_train)
y_test = label_to_idx_f(labels_test)
y_val = label_to_idx_f(labels_val)


### Models

Build a RNN baseline based on LSTM

In [6]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

def get_rnn_model(input_shape, out_dim, vocab_dim):
  input = Input(input_shape)

  embedding_layer = Embedding(input_dim = vocab_dim, output_dim = 64)(input)
  lstm_1 = LSTM(128, return_sequences = True,
                recurrent_dropout = 0.2)(embedding_layer)
  lstm_2 = LSTM(64, dropout = 0.2)(lstm_1)

  output = Dense(out_dim)(lstm_2)

  model = Model(input, output)
  model.compile(loss = SparseCategoricalCrossentropy(from_logits=True), 
                optimizer = Adam(1e-3), metrics=['accuracy'])
  model.summary()

  return model


In [7]:
# Use callback functions when building and experimenting the model 
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', 
                                            mode = 'max', patience = 5, 
                                            restore_best_weights = True)

In [8]:
model = get_rnn_model((20,), 2, vocab_dim)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 embedding (Embedding)       (None, 20, 64)            640000    
                                                                 
 lstm (LSTM)                 (None, 20, 128)           98816     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 788,354
Trainable params: 788,354
Non-trainable params: 0
_________________________________________________________________


In [9]:
history = model.fit(x_train, y_train, epochs = 10, 
                    validation_data = (x_val, y_val), 
                    callbacks = [callback])

Epoch 1/10
44/44 [==============================] - 22s 286ms/step - loss: 0.6271 - accuracy: 0.7041 - val_loss: 0.7585 - val_accuracy: 0.6233
Epoch 2/10
44/44 [==============================] - 8s 185ms/step - loss: 0.5467 - accuracy: 0.7384 - val_loss: 0.9725 - val_accuracy: 0.6837
Epoch 3/10
44/44 [==============================] - 6s 128ms/step - loss: 0.1822 - accuracy: 0.9350 - val_loss: 0.9222 - val_accuracy: 0.6884
Epoch 4/10
44/44 [==============================] - 7s 150ms/step - loss: 0.0464 - accuracy: 0.9878 - val_loss: 1.3865 - val_accuracy: 0.6605
Epoch 5/10
44/44 [==============================] - 5s 104ms/step - loss: 0.0192 - accuracy: 0.9943 - val_loss: 1.6604 - val_accuracy: 0.6419
Epoch 6/10
44/44 [==============================] - 7s 152ms/step - loss: 0.0098 - accuracy: 0.9964 - val_loss: 1.8819 - val_accuracy: 0.6744
Epoch 7/10
44/44 [==============================] - 7s 158ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.0212 - val_accuracy: 0.6791
Epoch

Datasets so small (1000 ca elements) tend to overfit, because train and validation are pretty similar. 

In [10]:
toLabels = np.vectorize(lambda e:'OGG' if e == 0 else 'SOGG')

def evaluate_model(model, x_test, y_test):
  """
  Args:
    - model: the model to use to make the prediction.
    - x_test: the sentences to label.
    - y_test: the actual labels.
  Returns:
    - The results of the evaluation of the model.

  """
  y_pred = np.argmax(model.predict(x_test), axis = -1)
  print(classification_report(toLabels(y_test), y_pred = toLabels(y_pred)))

In [11]:
evaluate_model(model, x_test, y_test)

8/8 [==============================] - 1s 8ms/step
              precision    recall  f1-score   support

         OGG       0.81      0.66      0.73       152
        SOGG       0.50      0.69      0.58        75

    accuracy                           0.67       227
   macro avg       0.66      0.68      0.66       227
weighted avg       0.71      0.67      0.68       227



### Bidirectional RNNs

For sequences other than time series (e.g. text), it is often the case that a RNN model
can perform better if it not only processes sequence from start to end, but also
backwards. For example, to predict the next word in a sentence, it is often useful to
have the context around the word, not only just the words that come before it.

Keras provides an easy API for you to build such **bidirectional RNNs**: the
`keras.layers.Bidirectional` wrapper.

Always worth a try to add the bidirectional layer.

In [13]:
from tensorflow.keras.layers import Bidirectional, TimeDistributed

def get_rnn_model_bd(input_shape, out_dim, vocabulary_length):
  input = Input(shape=input_shape)

  embedding_layer = Embedding(input_dim=vocabulary_length, output_dim=64)(input)

  lstm_1 = Bidirectional(LSTM(128, return_sequences=True,
                              recurrent_dropout = 0.2))(embedding_layer)

  lstm_2 = Bidirectional(LSTM(64, dropout = 0.2))(lstm_1)

  output = Dense(out_dim)(lstm_2)

  model = Model(input, output)

  model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
                optimizer = Adam(1e-3), metrics = ['accuracy'])
  
  model.summary()
  
  return model

In [15]:
model_bd = get_rnn_model_bd((20,), 2, vocab_dim)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 20, 64)            640000    
                                                                 
 bidirectional (Bidirectiona  (None, 20, 256)          197632    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1,002,242
Trainable params: 1,002,242
Non-tra

As we can see, the output of the bidirectional layer has doubled the output of the lstm layer seen before. (left to right and right to left).

In [16]:
history = model_bd.fit(x_train, y_train, epochs=10, validation_data = (x_val, y_val), callbacks = [callback])

Epoch 1/10
44/44 [==============================] - 23s 325ms/step - loss: 0.6139 - accuracy: 0.7098 - val_loss: 0.6990 - val_accuracy: 0.6233
Epoch 2/10
44/44 [==============================] - 12s 280ms/step - loss: 0.3966 - accuracy: 0.8127 - val_loss: 0.7842 - val_accuracy: 0.7209
Epoch 3/10
44/44 [==============================] - 9s 217ms/step - loss: 0.0959 - accuracy: 0.9628 - val_loss: 0.9848 - val_accuracy: 0.6744
Epoch 4/10
44/44 [==============================] - 9s 213ms/step - loss: 0.0341 - accuracy: 0.9921 - val_loss: 1.3633 - val_accuracy: 0.6605
Epoch 5/10
44/44 [==============================] - 8s 171ms/step - loss: 0.0176 - accuracy: 0.9929 - val_loss: 1.3652 - val_accuracy: 0.6558
Epoch 6/10
44/44 [==============================] - 9s 198ms/step - loss: 0.0073 - accuracy: 0.9971 - val_loss: 1.8712 - val_accuracy: 0.6419
Epoch 7/10
44/44 [==============================] - 9s 209ms/step - loss: 6.5562e-04 - accuracy: 1.0000 - val_loss: 1.7077 - val_accuracy: 0.6698


In [17]:
evaluate_model(model_bd, x_test, y_test)

8/8 [==============================] - 1s 17ms/step
              precision    recall  f1-score   support

         OGG       0.81      0.72      0.76       152
        SOGG       0.54      0.65      0.59        75

    accuracy                           0.70       227
   macro avg       0.67      0.69      0.68       227
weighted avg       0.72      0.70      0.71       227



## Transformers

Transformers are deep neural networks that over the last years achieved state of the art performances in several tasks.

Transformers replaces CNNs and RNNs with [self-attention](https://developers.google.com/machine-learning/glossary#self-attention). Self attention allows Transformers to easily transmit information across the input sequences.

For the transformer model implementation we will rely on a Python library called `transformers`, which provides an API inteface to several pre-trained models for fine-tuning or transfer-learning purposes.

In [18]:
!pip3 install transformers 
from transformers import AutoTokenizer, TFBertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.8 MB/s eta 0:00:00


### Pre-trained model

For this task we will use a pre-trained language model called [AlBERTo](github.com/marcopoli/AlBERTo-it). AlBERTo is a BERT model trained for the Italian language. In particular, AlBERTo is focused on the language used in social networks, specifically on Twitter. Due to the language and the type of data present in the dataset AlBERTo is the best fit for this kind of task.

You can find the pre-trained model in [huggingface](https://huggingface.co/bert-base-multilingual-cased?text=mi+piace+il+%5BMASK%5D), which is an open repository for pre-trained architectures, available in both pytorch and tensorflow (depending on the developers).

### Data pre-processing

Transoformers must recieve input in a standard format, namely divided in `input_ids`, `token_type_ids`, `attention_mask`.

In [19]:
def prepare_data_bert(x, y, maxSentenceLen = maxSentenceLen):
  """
  Args:
    - x: the sentences to label.
    - y: the actual labels.
    - maxSentenceLen: The maximum length of the sentences, it is used as a truncation length
  Returns:
    - A tuple with the input to feed into a transformers model, namely ((input_ids, attention_mask, token_type_ids), categorical_labels).

  """
  pad = tf.keras.preprocessing.sequence.pad_sequences
  tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
  dataFields = {
          "input_ids": [],
          "token_type_ids": [],
          "attention_mask": [],
          "subjectivity": []
      }
  lbls = {
      'SOG' : 1.0,
      'OGG' : 0.0
  }
  for i in range(len(x)):
      data = tokenizer(x[i])
      padded = pad([data['input_ids'], data['attention_mask'], data['token_type_ids']], padding = 'post', maxlen = maxSentenceLen)
      dataFields['input_ids'].append(padded[0])
      dataFields['attention_mask'].append(padded[1])
      dataFields['token_type_ids'].append(padded[-1])
      dataFields['subjectivity'].append(lbls[y[i]])
  
  for key in dataFields:
      dataFields[key] = np.array(dataFields[key])
  
  return [dataFields["input_ids"], dataFields["token_type_ids"], dataFields["attention_mask"]], dataFields["subjectivity"]

In [20]:
x_train_bert, y_train_bert = prepare_data_bert(sentences_train, labels_train)
x_val_bert, y_val_bert = prepare_data_bert(sentences_val, labels_val)
x_test_bert, y_test_bert = prepare_data_bert(sentences_test, labels_test)

### Model

In [21]:
def create_transformers_model(input_shape, out_dim):
  input_ids = Input(shape = input_shape, dtype = tf.int32)
  token_type_ids = Input(shape = input_shape, dtype = tf.int32)
  attention_mask = Input(shape = input_shape, dtype = tf.int32)

  bertModel = TFBertModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[-1]

  output = Dense(out_dim, activation=tf.nn.sigmoid)(tf.keras.layers.Dropout(0.1)(bertModel))

  model = Model(inputs = [input_ids, token_type_ids, attention_mask], outputs = output)
  model.compile(optimizer = Adam(1e-5), loss=SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])
  
  return model

In [23]:
bert_model = create_transformers_model((20,), 2)

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [24]:
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  184345344   ['input_3[0][0]',                
                                thPoolingAndCrossAt               'input_5[0][0]',          

In [25]:
history = bert_model.fit(x_train_bert, 
                         y_train_bert, 
                         epochs=4, 
                         validation_data = (x_val_bert, y_val_bert), 
                         batch_size = 16, 
                         callbacks = [callback]
                         )

Epoch 1/4
88/88 [==============================] - 82s 281ms/step - loss: 0.5839 - accuracy: 0.7084 - val_loss: 0.5753 - val_accuracy: 0.7442
Epoch 2/4
88/88 [==============================] - 15s 175ms/step - loss: 0.5007 - accuracy: 0.8163 - val_loss: 0.5624 - val_accuracy: 0.7302
Epoch 3/4
88/88 [==============================] - 13s 143ms/step - loss: 0.4378 - accuracy: 0.8735 - val_loss: 0.5508 - val_accuracy: 0.7581
Epoch 4/4
88/88 [==============================] - 11s 123ms/step - loss: 0.4055 - accuracy: 0.9099 - val_loss: 0.5517 - val_accuracy: 0.7535


In [26]:
evaluate_model(bert_model, x_test_bert, y_test_bert)

8/8 [==============================] - 3s 60ms/step
              precision    recall  f1-score   support

         OGG       0.77      0.84      0.80       152
        SOGG       0.60      0.51      0.55        75

    accuracy                           0.73       227
   macro avg       0.69      0.67      0.68       227
weighted avg       0.72      0.73      0.72       227

